In [39]:
# Import necessary libraries
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, rand
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import hash, col
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import col, abs

In [ ]:
# Step 1: Initialize Spark Session
#spark = SparkSession.builder \
 #   .appName("MovieRecommendation") \
 #   .getOrCreate()

In [40]:
import os
# Find the latest version of spark 3.x  from https://downloads.apache.org/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.5.5'
spark_version = 'spark-3.5.5'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 257 kB in 1s (182 kB/s)
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [43]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("sparkDates").getOrCreate()

In [44]:
# Step 2: Load the dataset (replace with your actual dataset path)

df = spark.read.json("/content/movie_results.json")

In [45]:
# Step 3: Data Cleaning & Filtering
# Step 2: Data Preprocessing

df_filtered = df.select("id", "title", "vote_average", "vote_count").filter(
    (col("vote_average").isNotNull()) & (col("vote_count") >= 50)
)
    # Check the first few rows of the filtered data
df_filtered.show(5)

+-------+--------------------+------------+----------+
|     id|               title|vote_average|vote_count|
+-------+--------------------+------------+----------+
| 950396|           The Gorge|       7.775|      1813|
|1126166|         Flight Risk|       6.086|       429|
|1064213|               Anora|       7.066|      1601|
| 762509|Mufasa: The Lion ...|         7.5|      1601|
|1241982|             Moana 2|         7.2|      1809|
+-------+--------------------+------------+----------+
only showing top 5 rows



In [46]:
# Step 3: Simulate Users for ALS Model
# Create user_id by hashing the movie_id to ensure overlap and make it positive
df_filtered = df_filtered.withColumn("user_id", abs((hash("id") % 1000)).cast(IntegerType()))

# Check the first few rows to ensure the user_id is created
df_filtered.select("user_id", "id", "vote_average").show(5)

+-------+-------+------------+
|user_id|     id|vote_average|
+-------+-------+------------+
|    610| 950396|       7.775|
|    671|1126166|       6.086|
|    860|1064213|       7.066|
|    718| 762509|         7.5|
|    579|1241982|         7.2|
+-------+-------+------------+
only showing top 5 rows



In [47]:
# Step 4: Prepare Data for ALS (ALS expects columns: user_id, movie_id, rating)
# Step 4: Prepare Data for ALS (ALS expects columns: user_id, movie_id, rating)
df_ratings = df_filtered.withColumnRenamed("vote_average", "rating") \
                        .select("user_id", "id", "rating")

In [48]:
import random

# Get distinct users
users = df_ratings.select("user_id").distinct().collect()

# Randomly shuffle the users to get a mix of train and test users
random.seed(42)
random.shuffle(users)

# Split the users into training (80%) and testing (20%) with some overlap
train_users = users[:int(0.8 * len(users))]  # 80% for training
test_users = users[int(0.7 * len(users)):]   # 20% for testing (with overlap from 70% onward)

# Create lists of user_ids for train and test sets
train_user_list = [user['user_id'] for user in train_users]
test_user_list = [user['user_id'] for user in test_users]

# Filter original data for train and test based on users
train_data = df_ratings.filter(df_ratings["user_id"].isin(train_user_list))
test_data = df_ratings.filter(df_ratings["user_id"].isin(test_user_list))

# Manually ensure overlap: Add some random user-movie pairs from the test set to the train set
overlap_size = 100  # Number of overlapping pairs
overlap_pairs = test_data.orderBy("user_id", "id").limit(overlap_size)

# Merge the overlap pairs with the train_data
train_data_with_overlap = train_data.union(overlap_pairs)

# Verify overlap again
train_pairs = train_data_with_overlap.select("user_id", "id").distinct().collect()
test_pairs = test_data.select("user_id", "id").distinct().collect()

train_set = set([(row['user_id'], row['id']) for row in train_pairs])
test_set = set([(row['user_id'], row['id']) for row in test_pairs])

# Find the overlap between train and test sets
overlap = train_set.intersection(test_set)
print(f"Overlapping user-movie pairs: {len(overlap)}")


Overlapping user-movie pairs: 717


In [49]:
# Step 2: Check for overlapping user-movie pairs between train and test
# Step 7: Handle Overlapping User-Movie Pairs between Train and Test Data
train_user_movie_pairs = train_data.select("user_id", "id").distinct()
test_user_movie_pairs = test_data.select("user_id", "id").distinct()

overlapping_pairs = train_user_movie_pairs.join(test_user_movie_pairs, on=["user_id", "id"], how="inner")

print(f"Overlapping user-movie pairs: {overlapping_pairs.count()}")


Overlapping user-movie pairs: 669


In [50]:
# Show the column names in both DataFrames
print("df_ratings columns:", df_ratings.columns)
print("df_filtered columns:", df_filtered.columns)

df_ratings columns: ['user_id', 'id', 'rating']
df_filtered columns: ['id', 'title', 'vote_average', 'vote_count', 'user_id']


In [51]:
# Step 1: Perform inner join between df_ratings and df_filtered on movie 'id'
common_movies = df_ratings.join(df_filtered, 'id', 'inner')

# Step 2: Resolve column name ambiguity
# Since both df_ratings and df_filtered have 'user_id', let's rename 'user_id' in df_filtered
df_filtered = df_filtered.withColumnRenamed('user_id', 'movie_user_id')

# Join again after renaming to avoid ambiguity
common_movies = df_ratings.join(df_filtered, 'id', 'inner')

# Step 3: Select only relevant columns
common_movies = common_movies.select(
    'user_id', 'title', 'rating', 'vote_average', 'vote_count'
)

# Step 4: Show the common movies
common_movies.show(10)

+-------+--------------------+------+------------+----------+
|user_id|               title|rating|vote_average|vote_count|
+-------+--------------------+------+------------+----------+
|    610|           The Gorge| 7.775|       7.775|      1813|
|    671|         Flight Risk| 6.086|       6.086|       429|
|    860|               Anora| 7.066|       7.066|      1601|
|    718|Mufasa: The Lion ...|   7.5|         7.5|      1601|
|    579|             Moana 2|   7.2|         7.2|      1809|
|    328|Sonic the Hedgehog 3|   7.7|         7.7|      2123|
|    599|Captain America: ...| 6.156|       6.156|       967|
|      2|                Flow| 8.298|       8.298|      1461|
|    660|              Amaran|   7.4|         7.4|       188|
|    431|           Mickey 17| 7.039|       7.039|       384|
+-------+--------------------+------+------------+----------+
only showing top 10 rows



In [52]:
# Step 5: Analyzing the ratings
# Find the average rating per movie
avg_ratings = common_movies.groupBy('title').agg({'rating': 'avg'}).withColumnRenamed('avg(rating)', 'avg_rating')
avg_ratings.show(10)

# Step 6: Recommend top 5 movies based on highest average rating
top_movies = avg_ratings.orderBy(col('avg_rating').desc()).limit(5)
top_movies.show()

+--------------------+----------+
|               title|avg_rating|
+--------------------+----------+
|Raya and the Last...|     7.841|
|Penguins of Madag...|     6.525|
|            Warcraft|      6.38|
|            Poseidon|     5.856|
|     My Name Is Khan|       8.0|
|  The Last Airbender|     4.637|
|My Cousin the Sex...|     6.938|
|          The Ritual|     6.257|
|Before I Go to Sleep|       6.5|
|    The Last Warrior|       6.9|
+--------------------+----------+
only showing top 10 rows

+--------------------+----------+
|               title|avg_rating|
+--------------------+----------+
|                Nude|       9.5|
|            Succubus|       9.5|
|           Kill Shot|     9.201|
|The Shawshank Red...|     8.708|
|       The Godfather|     8.689|
+--------------------+----------+



In [53]:
print(f"Train data user-movie pairs count: {train_user_movie_pairs.count()}")
print(f"Test data user-movie pairs count: {test_user_movie_pairs.count()}")

Train data user-movie pairs count: 5087
Test data user-movie pairs count: 1872


In [54]:
# Step 6: Build ALS Model
als = ALS(rank=10, maxIter=10, regParam=0.1, userCol="user_id", itemCol="id", ratingCol="rating", coldStartStrategy="drop")
model = als.fit(train_data)

In [55]:
# Step 8: Generate Predictions
predictions = model.transform(test_data).na.drop()
print(f"Total predictions: {predictions.count()}")

Total predictions: 669


In [56]:
# If there are no predictions, print a warning
# Step 9: Show Prediction Data
print(f"Total predictions: {predictions.count()}")

# Corrected column names: Use 'id' for movie_id instead of 'movie_id'
predictions.select("id", "user_id", "rating", "prediction").show(10, truncate=False)

Total predictions: 669
+-------+-------+------+----------+
|id     |user_id|rating|prediction|
+-------+-------+------+----------+
|1029281|471    |6.928 |6.882636  |
|495764 |471    |6.905 |6.859787  |
|845111 |471    |6.437 |6.3948507 |
|929    |471    |5.6   |5.563331  |
|859    |471    |7.199 |7.151861  |
|1685   |471    |6.144 |6.1037693 |
|960481 |471    |7.2   |7.152854  |
|340382 |737    |6.1   |6.060641  |
|3093   |451    |5.005 |4.974099  |
|22881  |451    |7.672 |7.6246324 |
+-------+-------+------+----------+
only showing top 10 rows



In [57]:
# Step 10: Evaluate Model Performance
if predictions.count() > 0:
    evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE): {rmse}")
else:
    print("RMSE cannot be computed as no predictions were generated.")

Root Mean Squared Error (RMSE): 0.04274500234148901


In [58]:
print(df_filtered.columns)

['id', 'title', 'vote_average', 'vote_count', 'movie_user_id']


In [59]:
df_ratings.columns

['user_id', 'id', 'rating']

In [61]:
from pyspark.sql.functions import col, explode

# Function to get movie recommendations
def get_movie_recommendations(user_id, num_recommendations=5):
    # Get the top N movie recommendations for the given user
    user_recs = model.recommendForAllUsers(num_recommendations)

    # Filter for the specific user
    user_recs = user_recs.filter(col("user_id") == user_id)

    # Explode the recommendations array to get each movie_id and rating separately
    user_recs_flat = user_recs.withColumn("recommendation", explode(col("recommendations"))) \
                               .select("user_id", "recommendation.id", "recommendation.rating")

    # Join with the movie details (from df_filtered) to get the titles, vote_average, etc.
    movie_details = df_filtered.select("id", "title", "vote_average", "vote_count")

    # Join user recommendations with movie details on 'id' (movie ID)
    recommendations_with_details = user_recs_flat.join(movie_details, user_recs_flat.id == movie_details.id, "inner")

    # Show the top N recommendations with movie details (title, rating, vote_average, etc.)
    recommendations_with_details.select("title", "rating", "vote_average", "vote_count").show(num_recommendations)

# Example usage: Get top 5 recommendations for user with user_id = 1
get_movie_recommendations(1)



+--------------------+---------+------------+----------+
|               title|   rating|vote_average|vote_count|
+--------------------+---------+------------+----------+
|New Gods: Nezha R...| 8.031101|       8.078|       449|
|        Spider-Man 2|7.2456875|       7.288|     15347|
|          The Father| 7.158721|       8.117|      3264|
|              8 Mile| 7.095564|       7.137|      7369|
|              Legend|7.0150347|       7.056|      3932|
+--------------------+---------+------------+----------+

